<a href="https://colab.research.google.com/github/puaqieshang/automatic-labeling-heart-vessels/blob/master/lstm_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Labeling Coronary Arteries using LSTM 

Update from previous version:

* Scaled data using StandardScaler() 
* Otherwise use Spherical Transform 


This version is able to run remotely on NCI


Before continuing, please refer to this [thread](https://stackoverflow.com/questions/38714959/understanding-keras-lstms/50235563#50235563) followed by this [thread](https://stackoverflow.com/questions/53955093/doubts-regarding-understanding-keras-lstms) to understand LSTM better 

In [1]:
# Import modules
import tensorflow
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import pandas as pd
import numpy as np
import os
import random

import tensorflow as tf
tf.__version__

# Does keras support cuda?

'2.2.0'

# Data Retrieval

Different Classes of Vessels and its labels
<table>
  <tr>
    <th>Label</th>
    <th>Class (Vessels)</th>
  </tr>
  <tr>
    <td>0</td>
    <td>LAD</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Diagonals</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Septals</td>
  </tr>
    <tr>
    <td>3</td>
    <td>LCX</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Obtuse Marginal</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Atrials</td>
  </tr>
    <tr>
    <td>6</td>
    <td>LCIM</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Acutes</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Crux</td>
  </tr>
</table>

## Import from Google Drive

In [2]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

# Add exclaimation mark to type code in terminal
!unzip -uq "/content/drive/My Drive/processed-rnn.zip"
!ls "/content/" # Lists all the files 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
drive  processed  sample_data


## Store Data on Pandas

There are 2 methods to store data:

1.   Load data from a **single** text file/heart e.g. AHG0004.txt then feed it into the model. Use a for loop to repeat for other text files
2.   Load data from **all** text files in insert into a single pandas dataframe 

After loading the data, it is recommended to scale data for MLP, check out the different scaling methods [here](https://towardsdatascience.com/normalization-vs-standardization-quantitative-analysis-a91e8a79cebf).

---
Edit:
*   ~~Training model with all heart datasets using option 2~~
*   ~~Testing model with a **single** dataset using option 1~~
*   Now both train and test uses Option 1, its like the model is learning from 324 different datasets 

### Option 1 - Single Dataset

In [0]:
from sklearn.preprocessing import StandardScaler 

def load_data_single(input_file, scalerType):
    
    # pri nt ('Loading data...')
    df = pd.read_csv(input_file, header=None, sep=' ', usecols=[1,2,3,4,5], 
                     names=['x','y', 'z', 'radius', 'labels'])

    # train_size = int(len(df) * (1 - test_split))
    
    # df = df.sample(frac=1)
    # df = df.sample(frac=1).reset_index(drop=True) # shuffle and reset row index
    # print(df.head())

    X = df[['x','y', 'z', 'radius']].values
    # print(X)
    df['labels'] = df['labels'].astype(int) #convert to int
    y = np.array(df['labels'].values)

    if scalerType is "spherical":
        print("Gimme a sec")

    else: # Standard Scaling
        scaler = StandardScaler() # can test other scaling methods, would not recommend normalisation
        scaler = scaler.fit(X) 

    standardized_X = scaler.transform(X)
    # print(standardized_X)

    # X_test = np.array(df['sequence'].values[train_size:])
    # print(y_test)
    y = to_categorical(y, num_classes=9)
    # X = pad_sequences(standardized_X) # SHOULD I REMOVE THIS?
    # print(X)

    return standardized_X, y


### Option 2 - Entire dataset
Combining all heart datasets into a single pandas dataframe

In [0]:
def load_data_all(folder_path):
    
    print ('Loading data...')
    columns = ['x','y', 'z', 'radius', 'labels']
    df = pd.DataFrame(columns=columns)

    folder = os.listdir(folder_path)
    # counter = 1

    for singleFile in folder:
        
        # print(f"{counter}. Currently retrieving data from {singleFile}")
        single_heart_df = pd.read_csv(dest+singleFile, header=None, 
                                      sep=' ', usecols=[1,2,3,4,5], 
                                      names=columns)

        # train_size = int(len(df) * (1 - test_split))
        # counter += 1
        df = df.append(single_heart_df, ignore_index=True) 
        
    # df = df.sample(frac=1).reset_index(drop=True) # shuffle and reset row index
    print(f"The dataset has a shape of {df.shape}\n")
    
    print(df.head())
    X = df[['x','y', 'z', 'radius']].values
    df['labels'] = df['labels'].astype(int) #convert to int
    y = np.array(df['labels'].values)
    # print(X_train.shape)
    # print(y_train)
    # X_test = np.array(df['sequence'].values[train_size:])
    # y_test = np.array(df['target'].values[train_size:])

    y = to_categorical(y, num_classes=9)
    X = pad_sequences(X)
    # X = np.reshape(X, (1, X.shape[0], X.shape[1]))

    return X, y


## Create Dataset
There are 2 ways of doing this - I'm not sure which one is more superior:

1.   Split ~~entire~~ a single dataframe into overlapping sequence or sliding windows
2.   [Divide the ~~entire~~ single heart dataframe into non-overlapping sequence](https://machinelearningmastery.com/prepare-univariate-time-series-data-long-short-term-memory-networks/)


Edit: This [thread](https://https://datascience.stackexchange.com/questions/27628/sliding-window-leads-to-overfitting-in-lstm) explains perfectly. Going for option 1

In [0]:
def create_dataset(X_dataset, y_dataset, look_back): # look_back = how many time steps
    X, y = [], []
    
    for i in range (len(X_dataset)-look_back):
        
        X.append(X_dataset[i:(i+look_back)])
        y.append(y_dataset[i+look_back])
    return np.array(X), np.array(y)

In [5]:
input_file = "/content/processed/AP1113.txt"

X_train, y_train = load_data_single(input_file, "standard")

print("Before creating dataset")
print(X_train.shape)
print(y_train.shape)
print(y_train[0:9])

Before creating dataset
(20429, 4)
(20429, 9)
[[0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1.]]


Creates a sequence of 20 points

In [6]:
SEQ_LENGTH = 20
look_back = SEQ_LENGTH
X_train, y_train = create_dataset(X_train, y_train, look_back)
print("After creating dataset")
print(X_train.shape)
print(y_train.shape)
print(y_train[0:1])

After creating dataset
(20409, 20, 4)
(20409, 9)
[[0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [7]:
print(X_train[0])

[[-0.20530254 -0.61929728  0.75777617  0.56862264]
 [-0.20665676 -0.62014886  0.75730993  0.56873322]
 [-0.20800768 -0.62099581  0.75684368  0.56895437]
 [-0.20936191 -0.62184276  0.75637744  0.56906494]
 [-0.21071613 -0.62268971  0.75591119  0.56928609]
 [-0.21206706 -0.62354128  0.75544495  0.56950724]
 [-0.21342128 -0.62438823  0.7549787   0.56972839]
 [-0.2147722  -0.62523518  0.75451246  0.56994954]
 [-0.21612643 -0.62608213  0.75404621  0.57017069]
 [-0.21748065 -0.62693371  0.75357997  0.57039184]
 [-0.21883157 -0.62778066  0.75311372  0.57061299]
 [-0.21986211 -0.62802595  0.75319849  0.57083414]
 [-0.22089264 -0.62826199  0.75324088  0.57105529]
 [-0.22192317 -0.62851191  0.75328327  0.57116586]
 [-0.2229504  -0.62877571  0.75336804  0.57138701]
 [-0.22397763 -0.6290534   0.75341042  0.57160816]
 [-0.22500156 -0.6293496   0.75345281  0.57182931]
 [-0.22602218 -0.62965969  0.75353758  0.57205046]
 [-0.2270395  -0.62999291  0.75357997  0.57227161]
 [-0.22805352 -0.63034002  0.75

# Model Definition

If not sure what the input shape should be in the first layer, refer to these websites:
*   [1D input for Time Series Prediction](https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/)
*   [2D input](https://stackoverflow.com/questions/46207004/keras-lstm-dense-layer-multidimensional-input)

Should the first layer be Dense or LSTM?
[Answer](https://github.com/keras-team/keras/issues/2673)

Edit 28/5/2020: Reduced number of Bidirectional layers


In [0]:
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [0]:
def create_model(look_back):
    print ('Creating model...')
    model = Sequential()
    # as first layer in a sequential model:
    #(batchsize, timesteps, features)

    model.add(Bidirectional(CuDNNLSTM(128, return_sequences=True),
                            input_shape=(look_back, 4))) 
    model.add(Dropout(0.2))
    model.add(Bidirectional(CuDNNLSTM(128, return_sequences=False))) 
    model.add(Dropout(0.2))

    model.add(Dense(64))
    # model.add(Bidirectional(CuDNNLSTM(100, return_sequences=True)))
    # model.add(Bidirectional(LSTM(output_dim=64, activation='sigmoid', inner_activation='hard_sigmoid')))
    # model.add(Dropout(0.2))

    model.add(Dense(32))
    # model.add(Bidirectional(CuDNNLSTM(100, return_sequences=True)))
    # model.add(Bidirectional(LSTM(output_dim=32, activation='sigmoid', inner_activation='hard_sigmoid')))
    # model.add(Dropout(0.2))

    model.add(Dense(16))
    # model.add(Bidirectional(CuDNNLSTM(9, return_sequences=False)))
    # model.add(Bidirectional(LSTM(output_dim=16, activation='sigmoid', inner_activation='hard_sigmoid')))
    # model.add(Dropout(0.2))


    model.add(Dense(9, activation='softmax'))
    # model.add(Dense(1, activation='softmax'))
    

    print ('Compiling...')
    model.compile(loss='categorical_crossentropy',
                  optimizer='Adam',
                  metrics=['accuracy'])
    
    # model.build(input_shape)

    print(model.summary())
    return model

In [10]:
look_back = SEQ_LENGTH
model = create_model(look_back)

Creating model...
Compiling...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 20, 256)           137216    
_________________________________________________________________
dropout (Dropout)            (None, 20, 256)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               395264    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)          

# Train and Validate Model

Should I shuffle my data?

[Solution 1](https://stackoverflow.com/questions/44788946/shuffling-training-data-with-lstm-rnn):
You should shuffle the training data (a set of fixed-length sequences) when you shuffle the order in which sequences are fed to the RNN. YOU SHOULD NOT SHUFFLE THE ORDERING OF POINTS WITHIN A SEQUENCE - otherwise whats the point of using LSTM, you might as well use a MLP. 


Edit: Solved NCI issue. Now just wondering why does the val loss varies so much




In [12]:
# Case where data is shuffled
look_back = SEQ_LENGTH
NO_OF_SAMPLES = 300

def train_and_validate(input_folder_path):
    
    print("Training model...")
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    folder = os.listdir(input_folder_path)
    # print(len(folder))
    # counter = 1

    for counter in range(0, NO_OF_SAMPLES):
        i = random.randint(0, len(folder)-1) # random might be bad?
        input_file_path = input_folder_path+folder[i]
        print(f"{counter+1}. Now training on {folder[i]}")
    # for singleFile in folder:
    #     print(f"Heart No.{counter} - {singleFile}")
    #     input_file_path = input_folder_path + singleFile   
        X_train, y_train = load_data_single(input_file_path, "standard")
        # print(f"{counter}. Currently retrieving data from {singleFile}")
        X_train, y_train = create_dataset(X_train, y_train, look_back)


       # 0.1 rows are used for validation
        hist = model.fit(X_train, y_train, 
                        batch_size=64, epochs=1, 
                        shuffle = True, # shuffle the sequences, but not the individual data
                        validation_split = 0.1, verbose = 2)

        train_loss.append(hist.history['loss'][0])
        train_acc.append(hist.history['accuracy'][0])
        val_loss.append(hist.history['val_loss'][0])
        val_acc.append(hist.history['val_accuracy'][0])
        counter += 1

    print("Done Training.")
    return train_loss, train_acc, val_loss, val_acc


folder_path = "/content/processed/" # The only path that I have to change for NCI
train_loss, train_acc, val_loss, val_acc = train_and_validate(folder_path)

Training model...
1. Now training on AP1295.txt
80/80 - 10s - loss: 0.3516 - accuracy: 0.8623 - val_loss: 0.5217 - val_accuracy: 0.8322
2. Now training on AHG0050.txt
148/148 - 8s - loss: 0.4359 - accuracy: 0.8463 - val_loss: 0.0090 - val_accuracy: 1.0000
3. Now training on AP1291.txt
111/111 - 8s - loss: 0.4116 - accuracy: 0.8398 - val_loss: 5.2620 - val_accuracy: 0.0000e+00
4. Now training on AP1262.txt
41/41 - 7s - loss: 0.5327 - accuracy: 0.8252 - val_loss: 0.0094 - val_accuracy: 1.0000
5. Now training on AP1194.txt
161/161 - 9s - loss: 0.4253 - accuracy: 0.8277 - val_loss: 4.9662 - val_accuracy: 0.0000e+00
6. Now training on AP1219.txt
113/113 - 8s - loss: 0.2159 - accuracy: 0.9088 - val_loss: 4.0614 - val_accuracy: 0.0000e+00
7. Now training on AP1127.txt
94/94 - 8s - loss: 0.3488 - accuracy: 0.8588 - val_loss: 2.9217 - val_accuracy: 0.2889
8. Now training on AP1160.txt
198/198 - 10s - loss: 0.2990 - accuracy: 0.8680 - val_loss: 7.3138 - val_accuracy: 0.0000e+00
9. Now training o

KeyboardInterrupt: ignored

The code below trains on individual data - not really recommended?

In [0]:
# Case where data is shuffled
'''
look_back = SEQ_LENGTH
NO_OF_SAMPLES = 100

def train_and_validate(input_folder_path):
    
    print("Training model...")
    train_loss = []
    train_acc = []
    val_loss = []
    val_acc = []

    folder = os.listdir(input_folder_path)
    # print(len(folder))
    # counter = 1

    for counter in range(0, NO_OF_SAMPLES):
        i = random.randint(0, len(folder)-1) # random might be bad?
        input_file_path = input_folder_path+folder[i]
        print(f"{counter+1}. Now training on {folder[i]}")
    # for singleFile in folder:
    #     print(f"Heart No.{counter} - {singleFile}")
    #     input_file_path = input_folder_path + singleFile   
        X_train, y_train = load_data_single(input_file_path)
        # print(f"{counter}. Currently retrieving data from {singleFile}")
        X_train, y_train = create_dataset(X_train, y_train, look_back)


       # 0.1 rows are used for validation
        hist = model.fit(X_train, y_train, 
                        batch_size=16, epochs=1, 
                        shuffle = True, # shuffle the sequences, but not the individual data
                        validation_split = 0.1, verbose = 1)

        train_loss.append(hist.history['loss'][0])
        train_acc.append(hist.history['accuracy'][0])
        val_loss.append(hist.history['val_loss'][0])
        val_acc.append(hist.history['val_accuracy'][0])
        counter += 1

    print("Done Training.")
    return train_loss, train_acc, val_loss, val_acc


folder_path = "/content/processed/" # The only path that I have to change for NCI
train_loss, train_acc, val_loss, val_acc = train_and_validate(folder_path)
'''

## Plot Accuracy and Loss vs Number of Samples being trained

In [0]:
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd

style.use("ggplot")

def create_acc_loss_graph(train_loss, train_acc, test_loss, test_acc):
    epoch = list(range(0, NO_OF_SAMPLES, 1))

    d = {'epochs': epoch,
        'train_acc': train_acc,
        'train_loss': train_loss,
        'test_acc': val_acc, 
        'test_loss': val_loss}

    df = pd.DataFrame(d)

    df['train_acc_avg'] = df['train_acc'].ewm(alpha=.02).mean()  # exponential weighted moving average
    df['test_acc_avg'] = df['test_acc'].ewm(alpha=.02).mean()
    df['train_loss_avg'] = df['train_loss'].ewm(alpha=.02).mean()
    df['test_loss_avg'] = df['test_loss'].ewm(alpha=.02).mean()

    # Then plot using pandas:
    df.plot(x='epochs', y=['train_acc_avg', 'test_acc_avg'], figsize=(8,4))
    plt.xlabel("No of samples/epochs")
    plt.ylabel("Accuracy")
    df.plot(x='epochs', y=['train_loss_avg', 'test_loss_avg'], figsize=(8,4))
    plt.xlabel("No of samples/epochs")
    plt.ylabel("Loss")

    plt.show()

create_acc_loss_graph(train_loss, train_acc, val_loss, val_acc)

# export jpeg

NameError: ignored

In [0]:
# print(hist.history['accuracy'])

# Testing Model on a Single Dataset

In [0]:
# Unseen
input_file = "/content/processed/AP1113.txt" #bad accuracy

X_test, y_test = load_data_single(input_file, "standard")
X_test, y_test = create_dataset(X_test, y_test, look_back)
print("Testing on unseen dataset: \n")

# model.reset_states()
loss, acc = model.evaluate(X_test, y_test, batch_size=8, verbose = 1)

print('Test loss:', loss)
print('Test accuracy:', acc)

# Seen
input_file = "/content/processed/AP1173.txt" # the last thing that was trained

X_test, y_test = load_data_single(input_file, "standard")
X_test, y_test = create_dataset(X_test, y_test, look_back)

print("\nTesting on seen dataset: \n")
# model.reset_states()
loss, acc = model.evaluate(X_test, y_test, batch_size=8, verbose = 1)

print('Test loss:', loss)
print('Test accuracy:', acc)

Testing on unseen dataset: 

2552/2552 [==============================] - 19s 7ms/step - loss: 1.1280 - accuracy: 0.6530
Test loss: 1.1279670000076294
Test accuracy: 0.6529962420463562

Testing on seen dataset: 

2160/2160 [==============================] - 17s 8ms/step - loss: 3.7751 - accuracy: 0.3578
Test loss: 3.775066375732422
Test accuracy: 0.3577795922756195


In [0]:
# input_file = "/content/processed/AHG0004.txt" # unseen - terrible accuracy

input_file = "/content/processed/AP1173.txt" #seen data 

X_test, y_test= load_data_single(input_file, "standard")
X_test, y_test = create_dataset(X_test, y_test, look_back)

yhat = model.predict_classes(X_test, verbose=1)


# yhat is predicted, y_test is real 
# correctCount = np.sum(np.argmax(y_test, axis =-1) is yhat) # argmax is inverse of to_categorical()
y_test = np.argmax(y_test, axis =-1)
correctCount = 0

for i, j in zip(y_test, yhat):
    if i == j:
        correctCount += 1

print(f"Accuracy is {round(correctCount/len(y_test), 4)}.")


 

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
540/540 [==============================] - 3s 5ms/step
Accuracy is 0.3578.
